<a href="https://colab.research.google.com/github/yishairasowsky/Sefaria-Project/blob/master/ML/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd
# import pkl

In [16]:
# Here I tried to pick an encoding type that would accomodate the Hebrew characters. No luck yet.
encoding = "latin-1"
# encoding = 'unicode_escape'
# encoding = "ISO-8859-1"
# encoding = "ISO-8859-8"
# encoding = "cp1255"

full_df = pd.read_csv(
    '/content/drive/My Drive/Colab Datasets/topic_data.csv',
    encoding=encoding,
    # quoting=3,
    error_bad_lines=False,
    engine='python'
    # )[:5000]
    )

Skipping line 1099: field larger than field limit (131072)
Skipping line 1155: ',' expected after '"'
Skipping line 22113: unexpected end of data
Skipping line 1099: Expected 5 fields in line 1099, saw 12
Skipping line 1100: Expected 5 fields in line 1100, saw 10
Skipping line 1101: Expected 5 fields in line 1101, saw 9
Skipping line 1102: Expected 5 fields in line 1102, saw 9
Skipping line 1104: Expected 5 fields in line 1104, saw 13
Skipping line 1106: Expected 5 fields in line 1106, saw 7
Skipping line 1107: Expected 5 fields in line 1107, saw 9
Skipping line 1108: Expected 5 fields in line 1108, saw 13
Skipping line 1109: Expected 5 fields in line 1109, saw 19
Skipping line 1110: Expected 5 fields in line 1110, saw 9
Skipping line 1111: Expected 5 fields in line 1111, saw 13
Skipping line 1112: Expected 5 fields in line 1112, saw 6
Skipping line 1113: Expected 5 fields in line 1113, saw 7
Skipping line 1114: Expected 5 fields in line 1114, saw 13
Skipping line 1115: Expected 5 fiel

In [17]:
print("Original shape:",full_df.shape)

full_df.dropna(inplace=True)
print("Without nulls",full_df.shape)

full_df.drop_duplicates(inplace=True)
print("Without duplicates",full_df.shape)


Original shape: (22061, 5)
Without nulls (14535, 5)
Without duplicates (14535, 5)


In [18]:
full_df.head()


,Ref,En,He,Topics,Expanded Topics
0,Bekhorot 28a:5,Â§ The mishna teaches: If <b>a blemish develop...,× ××× ×× ××× ××ª×× ×©× ×ª× ×¨×©××...,laws-of-animal-firstlings,laws-of-animal-firstlings laws-of-worship-of-g...
1,Sforno on Genesis 21:22:1,"××××§×× ×¢××, seeing that Gâd clearly...",<b>××××× ×¢××</b> ××¤××× ×× × ××...,abraham-and-abimelech,entity specific-biblical-person-relationship r...
3,Gittin 78a:1,"<b>a modifier,</b> meaning that this board is ...",×©× ×××× ××× ×××ª ××× ×©× ×××...,dinei-haget,entity halakhah continuant laws mitzvot generi...
5,Sanhedrin 2a:5,The <b>intercalation of the month</b> is perfo...,×¢××××¨ ××××© ××©××©× ×¢××××¨ ××©...,laws-of-judges-and-courts judgements1 laws-of-...,beit-din group-of-people independent-continuan...
6,Pesachim 78a:15,<b>The statement of Rabbi Eliezer</b> is corre...,×××¨× ×¨×× ××××¢××¨ ×××××× ×©×...,torah-study-and-a-livelihood life,lifecycle-events entity philosophy religion to...


In [0]:
from bs4 import BeautifulSoup
import re
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))

def preprocess_text(sentence):

    # HTML decoding
    sentence = BeautifulSoup(sentence, "lxml").text 
    
    # lowercase text
    sentence = sentence.lower() 

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    # Removing stopwords
    sentence = ' '.join(word for word in sentence.split() if word not in STOPWORDS) # delete stopwors from text

    return sentence

clean_df = full_df[['En','Topics']]
clean_df.En = clean_df.En.apply(preprocess_text)
clean_df.head()

In [0]:
clean_df.to_pickle("/content/drive/My Drive/Colab Datasets/clean_df.pkl")

In [0]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")

def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

stem_df = clean_df
stem_df.En = stem_df.En.apply(stemming)
stem_df.head()

In [0]:
stem_df.to_pickle("/content/drive/My Drive/Colab Datasets/stem_df.pkl")

In [0]:
eng_df = stem_df[['En','Topics']]
topics_df = eng_df.join(eng_df.pop('Topics').str.get_dummies(' '))
topics_df.head()

In [0]:
topics_df.to_pickle("/content/drive/My Drive/Colab Datasets/topics_df.pkl")